In [1]:
#Passo1:Importar bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import time
import urllib.parse
import json
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
import random
import re
from collections import defaultdict
import threading

# --- PASSO 2: Definir a URL e fazer a requisição inicial ---
url = "https://www.drogariaveracruz.com.br/medicamentos/"
url_base = url

# Header para evitar bloqueio - OTIMIZADO
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'pt-BR,pt;q=0.8,en;q=0.6',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

# Criar sessão com pool de conexões otimizado
session = requests.Session()
session.headers.update(headers)
adapter = requests.adapters.HTTPAdapter(
    pool_connections=20,  # Mais conexões simultâneas
    pool_maxsize=50,     # Pool maior
    max_retries=3
)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Controle de rate limiting inteligente
last_request_time = 0
request_lock = threading.Lock()
consecutive_errors = 0

# Cache para evitar requisições duplicadas
url_cache = {}
cache_lock = threading.Lock()

# --- Funções auxiliares OTIMIZADAS ---

def achar_nome(div):
    tag_h2 = div.find('h2', class_='title')
    if tag_h2:
        tag_a = tag_h2.find('a')
        if tag_a:
            return tag_a.get_text(strip=True)
    return "Nome não encontrado"

def achar_preco(div):
    """OTIMIZADO: Busca o preço unitário"""
    tag_preco = div.find('p', class_="unit-price p-0")
    if tag_preco:
        return tag_preco.get_text().strip()
    return None

def achar_precopix(div):
    """OTIMIZADO: Busca o preço PIX com método mais eficiente"""
    # Método mais direto primeiro
    tag_precopix = div.find('p', class_="sale-price-pix")
    if tag_precopix:
        strong_tag = tag_precopix.find('strong')
        if strong_tag:
            return strong_tag.get_text().strip()
        return tag_precopix.get_text().strip()
    return None 

def achar_total_paginas(soup_inicial):
    container_template = soup_inicial.find('div', class_='page-template')
    if container_template:
        divs_candidatas = container_template.find_all('div', class_='text-center pt-3')
        for div in divs_candidatas:
            texto = div.get_text(strip=True)
            if "Página" in texto:
                # Regex mais eficiente
                match = re.search(r'Página\s+\d+\s+de\s+(\d+)', texto)
                if match:
                    return int(match.group(1))
                # Fallback para método original
                partes = texto.split()
                ultima_palavra = partes[-1]
                if ultima_palavra.isdigit():
                    return int(ultima_palavra)
    return 1

def achar_link(produto_html, url_base):
    tag_h2 = produto_html.find('h2', class_='title')
    if tag_h2:
        tag_a = tag_h2.find('a')
        if tag_a and 'href' in tag_a.attrs:
            link_relativo = tag_a.get('href')
            return urllib.parse.urljoin(url_base, link_relativo)
    return None

def achar_precodesconto(produto_html):
    """OTIMIZADO: Busca o preço com desconto"""
    tag_venda = produto_html.find('p', class_="sale-price p-0")
    if tag_venda:
        strong_tag = tag_venda.find('strong')
        return (strong_tag or tag_venda).get_text(strip=True)
    return None

def limpar_json_string(json_string):
    """NOVA FUNÇÃO: Limpa strings JSON malformadas"""
    if not json_string:
        return json_string
    
    # Operações mais eficientes em uma só passada
    json_string = re.sub(r'[\n\r\t]+', '', json_string)
    json_string = re.sub(r'\s{2,}', ' ', json_string)
    json_string = re.sub(r',,+', ',', json_string)
    json_string = re.sub(r',\s*([}\]])', r'\1', json_string)
    
    return json_string.strip()

def extrair_detalhes_do_json(soup_detalhes):
    """OTIMIZADO: Extração mais robusta do JSON-LD"""
    brand = None
    code = None
    
    # Buscar apenas o primeiro script JSON-LD relevante
    script_tag = soup_detalhes.find('script', type='application/ld+json')
    if not script_tag or not script_tag.string:
        return brand, code
            
    try:
        raw_json = script_tag.string.strip()
        cleaned_json = limpar_json_string(raw_json)
        dados_json = json.loads(cleaned_json)
        
        # Tratamento simplificado
        if isinstance(dados_json, list) and dados_json:
            for item in dados_json:
                if isinstance(item, dict) and ('brand' in item or 'gtin13' in item):
                    dados_json = item
                    break
        
        # Extração direta
        if isinstance(dados_json, dict):
            # Marca
            brand_data = dados_json.get('brand')
            if isinstance(brand_data, dict):
                brand = brand_data.get('name')
            elif isinstance(brand_data, str):
                brand = brand_data
            
            # Código
            code = dados_json.get('gtin13') or dados_json.get('gtin')
            if code:
                code = str(code)
                
    except:
        # Fallback com regex mais simples
        try:
            raw_json = script_tag.string
            brand_match = re.search(r'"brand":\s*(?:{\s*"name":\s*"([^"]+)"|"([^"]+)")', raw_json)
            if brand_match:
                brand = brand_match.group(1) or brand_match.group(2)
            
            gtin_match = re.search(r'"gtin(?:13)?":\s*"([^"]+)"', raw_json)
            if gtin_match:
                code = gtin_match.group(1)
        except:
            pass
    
    return brand, code

def extrair_detalhes_adicionais_da_pagina(soup_detalhes):
    """NOVA FUNÇÃO: Extrai detalhes adicionais da página do produto"""
    detalhes = {}
    
    # Buscar preços na página do produto (mais precisos)
    unit_price_elem = soup_detalhes.find('p', class_='unit-price')
    if unit_price_elem:
        detalhes['unit_price_detailed'] = unit_price_elem.get_text(strip=True)
    
    discount_price_elem = soup_detalhes.find('p', class_='sale-price')
    if discount_price_elem:
        detalhes['discount_price_detailed'] = discount_price_elem.get_text(strip=True)
    
    pix_price_elem = soup_detalhes.find('p', class_='sale-price-pix')
    if pix_price_elem:
        detalhes['pix_price_detailed'] = pix_price_elem.get_text(strip=True)
    
    # Buscar subcategoria
    breadcrumb = soup_detalhes.find(class_='breadcrumb')
    if breadcrumb:
        detalhes['sub_category'] = breadcrumb.get_text(strip=True)
    else:
        detalhes['sub_category'] = None
    
    # Buscar código do produto
    code_element = soup_detalhes.find('span', class_='product-code')
    if code_element:
        detalhes['product_code'] = code_element.get_text(strip=True)
    else:
        detalhes['product_code'] = None
    
    return detalhes

def limpar_preco(preco_str):
    """OTIMIZADO: Limpeza mais eficiente de preços"""
    if not preco_str: 
        return None
    try:
        # Regex mais eficiente para extrair números
        match = re.search(r'(\d{1,3}(?:\.\d{3})*(?:,\d{2})?)', preco_str.replace("R$", ""))
        if match:
            preco_limpo = match.group(1).replace('.', '').replace(',', '.')
            return float(preco_limpo)
    except:
        pass
    return None

def baixar_url(url, tentativas=2):
    """OTIMIZADO: Download mais inteligente com rate limiting adaptativo"""
    global last_request_time, consecutive_errors
    
    # Cache check
    with cache_lock:
        if url in url_cache:
            return url_cache[url]
    
    for i in range(tentativas):
        try:
            # Rate limiting inteligente
            with request_lock:
                current_time = time.time()
                time_since_last = current_time - last_request_time
                
                # Delay adaptativo baseado em erros
                if consecutive_errors > 0:
                    delay = min(0.5 + (consecutive_errors * 0.2), 2.0)
                else:
                    delay = 0.1  # Delay mínimo quando tudo está funcionando
                
                if time_since_last < delay:
                    time.sleep(delay - time_since_last)
                
                last_request_time = time.time()
            
            response = session.get(url, timeout=20)
            
            if response.status_code == 200:
                consecutive_errors = 0
                # Cache response
                with cache_lock:
                    url_cache[url] = response
                return response
            elif response.status_code == 429:
                consecutive_errors += 1
                time.sleep(min(2 ** i, 10))  # Exponential backoff
            else:
                consecutive_errors += 1
                
        except Exception as e:
            consecutive_errors += 1
            if i == tentativas - 1:
                print(f"⚠️ Falha persistente: {str(e)[:50]}...")
            time.sleep(random.uniform(0.5, 2))
    
    return None

def processar_produto_rapido(produto, url_base):
    """OTIMIZADO: Processamento mais rápido de produto"""
    nome = achar_nome(produto)
    preco = achar_preco(produto)
    precopix = achar_precopix(produto)
    precodesconto = achar_precodesconto(produto)
    link_produto = achar_link(produto, url_base)

    brand, code = None, None
    detalhes_extras = {}
    
    # Apenas processar se o link for válido
    if link_produto:
        responseprodutos = baixar_url(link_produto)
        if responseprodutos:
            soup_produto = BeautifulSoup(responseprodutos.content, 'html.parser')
            brand, code = extrair_detalhes_do_json(soup_produto)
            detalhes_extras = extrair_detalhes_adicionais_da_pagina(soup_produto)

    # Usar preços da página do produto se disponíveis (mais precisos)
    preco_final = detalhes_extras.get('unit_price_detailed') or preco
    precodesconto_final = detalhes_extras.get('discount_price_detailed') or precodesconto
    precopix_final = detalhes_extras.get('pix_price_detailed') or precopix

    unit_price = limpar_preco(preco_final)
    discount_price = limpar_preco(precodesconto_final)
    pix_price = limpar_preco(precopix_final)

    # Calcular descontos
    discount = round(unit_price - discount_price, 2) if unit_price and discount_price else None
    pix_discount = round(unit_price - pix_price, 2) if unit_price and pix_price else None

    return {
        'Nome': nome,
        'Marca': brand,
        'GTIN': code,
        'Codigo': detalhes_extras.get('product_code'),
        'Sub_categoria': detalhes_extras.get('sub_category'),
        'Preco_unitario': unit_price,
        'Preco_com_desconto': discount_price,
        'Preco_pix': pix_price,
        'Desconto': discount,
        'Desconto_com_pix': pix_discount,
        'Link': link_produto
    }

def processar_pagina_completa(pagina_info):
    """NOVA FUNÇÃO: Processa uma página completa de forma otimizada"""
    pagina, total_paginas, url_base = pagina_info
    
    print(f"📄 Processando página {pagina}/{total_paginas}")
    url_pagina = f'https://www.drogariaveracruz.com.br/medicamentos/?p={pagina}'
    
    response = baixar_url(url_pagina)
    if not response:
        print(f"❌ Falha ao carregar página {pagina}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    div_produtos = soup.find_all('div', class_='li')
    print(f"🔍 Página {pagina}: {len(div_produtos)} produtos encontrados")

    # Processar produtos da página com mais workers
    produtos_pagina = []
    with ThreadPoolExecutor(max_workers=8) as executor:  # 8 workers por página
        futures = [executor.submit(processar_produto_rapido, produto, url_base) 
                  for produto in div_produtos]
        
        for future in as_completed(futures):
            resultado = future.result()
            if resultado:
                produtos_pagina.append(resultado)

    print(f"✅ Página {pagina}: {len(produtos_pagina)} produtos processados")
    return produtos_pagina

# INÍCIO DO PROCESSAMENTO OTIMIZADO
start_time = time.time()

# --- PRIMEIRA PÁGINA para descobrir total ---
print("🔍 Descobrindo total de páginas...")
response = baixar_url(url)
soup = BeautifulSoup(response.content, 'html.parser')
total_paginas = achar_total_paginas(soup)
print(f"📊 Total de páginas encontradas: {total_paginas}")

# Preparar lista de páginas para processamento paralelo
max_paginas = min(20, total_paginas)  # Limite para teste inicial
paginas_info = [(p, total_paginas, url_base) for p in range(1, max_paginas + 1)]

lista_de_produtos = []

print(f"🚀 Iniciando extração paralela de {max_paginas} páginas...")
print(f"⏱️  Tempo de início: {time.strftime('%H:%M:%S')}")

# PROCESSAMENTO PARALELO DE PÁGINAS
with ThreadPoolExecutor(max_workers=4) as page_executor:  # 4 páginas em paralelo
    page_futures = [page_executor.submit(processar_pagina_completa, info) 
                   for info in paginas_info]
    
    for future in as_completed(page_futures):
        produtos_pagina = future.result()
        lista_de_produtos.extend(produtos_pagina)
        print(f"📊 Total acumulado: {len(lista_de_produtos)} produtos")

# Calcular tempo de execução
end_time = time.time()
tempo_execucao = end_time - start_time

# --- RESULTADOS FINAIS ---
print(f"\n🎉 Extração concluída em {tempo_execucao:.1f} segundos!")
print(f"📊 Coletados {len(lista_de_produtos)} produtos")
print(f"⚡ Velocidade: {len(lista_de_produtos)/tempo_execucao:.1f} produtos/segundo")

print("\n📊 Gerando tabela...")
df = pd.DataFrame(lista_de_produtos)
display(df)

print("💾 Salvando arquivo CSV...")
df.to_csv('veracruz_otimizado.csv', index=False)
print("✅ Arquivo salvo como 'veracruz_otimizado.csv'")

# Estatísticas de qualidade
print(f"\n📈 ESTATÍSTICAS DE QUALIDADE:")
print(f"Total de produtos: {len(df)}")
print(f"Produtos com preço unitário: {df['Preco_unitario'].notna().sum()}")
print(f"Produtos com preço desconto: {df['Preco_com_desconto'].notna().sum()}")
print(f"Produtos com preço PIX: {df['Preco_pix'].notna().sum()}")
print(f"Produtos com marca: {df['Marca'].notna().sum()}")
print(f"Produtos com GTIN: {df['GTIN'].notna().sum()}")

# Estatísticas de performance
print(f"\n⚡ ESTATÍSTICAS DE PERFORMANCE:")
print(f"Tempo total: {tempo_execucao:.1f}s")
print(f"Páginas processadas: {max_paginas}")
print(f"Produtos por página (média): {len(lista_de_produtos)/max_paginas:.1f}")
print(f"Tempo por página (média): {tempo_execucao/max_paginas:.1f}s")
print(f"Requests em cache: {len(url_cache)}")
print(f"Taxa de erro final: {consecutive_errors}")

# Exemplo de produto completo
produtos_completos = df[(df['Preco_unitario'].notna()) & 
                       (df['Marca'].notna()) & 
                       (df['GTIN'].notna())]

if len(produtos_completos) > 0:
    print(f"\n📋 Exemplo de produto com dados completos:")
    exemplo = produtos_completos.iloc[0]
    for coluna, valor in exemplo.items():
        if valor is not None and str(valor) != 'nan':
            print(f"  {coluna}: {valor}")
else:
    print(f"\n📋 Exemplo de produto coletado:")
    if len(df) > 0:
        exemplo = df.iloc[0]
        for coluna, valor in exemplo.items():
            if valor is not None and str(valor) != 'nan':
                print(f"  {coluna}: {valor}")

print(f"\n🏆 COMPARATIVO DE PERFORMANCE:")
print(f"Versão original: ~{max_paginas * 3:.0f}s estimados (sequencial)")
print(f"Versão otimizada: {tempo_execucao:.1f}s reais")
print(f"Melhoria de velocidade: {(max_paginas * 3)/tempo_execucao:.1f}x mais rápido!")
print(f"Concurrent operations: 4 pages × 8 products = 32 simultâneos")

🔍 Descobrindo total de páginas...
📊 Total de páginas encontradas: 214
🚀 Iniciando extração paralela de 20 páginas...
⏱️  Tempo de início: 15:19:59
📄 Processando página 1/214
📄 Processando página 2/214
📄 Processando página 3/214
📄 Processando página 4/214
📊 Total de páginas encontradas: 214
🚀 Iniciando extração paralela de 20 páginas...
⏱️  Tempo de início: 15:19:59
📄 Processando página 1/214
📄 Processando página 2/214
📄 Processando página 3/214
📄 Processando página 4/214
🔍 Página 1: 20 produtos encontrados
🔍 Página 1: 20 produtos encontrados
🔍 Página 4: 20 produtos encontrados
🔍 Página 2: 20 produtos encontrados
🔍 Página 4: 20 produtos encontrados
🔍 Página 2: 20 produtos encontrados
🔍 Página 3: 20 produtos encontrados
🔍 Página 3: 20 produtos encontrados
✅ Página 1: 20 produtos processados
📄 Processando página 5/214
📊 Total acumulado: 20 produtos
✅ Página 1: 20 produtos processados
📄 Processando página 5/214
📊 Total acumulado: 20 produtos
✅ Página 2: 20 produtos processados
📄 Processand

,Nome,Marca,GTIN,Codigo,Sub_categoria,Preco_unitario,Preco_com_desconto,Preco_pix,Desconto,Desconto_com_pix,Link
0,Blephagel 40g,União Química,3662042000249,None,None,129.90,109.83,109.83,20.07,20.07,https://www.drogariaveracruz.com.br/gel-para-h...
1,Renu Fresh Solução Multiuso 355ml + 120ml + Es...,Bausch &amp; Lomb,7896046381004,None,None,49.90,35.56,35.56,14.34,14.34,https://www.drogariaveracruz.com.br/solucao-mu...
2,Niquitin 21mg 7 adesivos transdérmicos transpa...,Gsk,7898928257032,None,None,132.44,100.07,100.07,32.37,32.37,https://www.drogariaveracruz.com.br/niquitin-2...
3,Aradois 50mg 30 comprimidos revestidos,Biolab Sanus,7896112486787,None,None,57.29,4.90,4.90,52.39,52.39,https://www.drogariaveracruz.com.br/aradois-50...
4,Probid 30 cápsulas,Apsen,7896637032209,None,None,88.90,84.46,84.46,4.44,4.44,https://www.drogariaveracruz.com.br/suplemento...
...,...,...,...,...,...,...,...,...,...,...,...
395,Rosucor 10mg 60 comprimidos revestidos,Torrent,8902220109582,None,None,84.36,56.10,56.10,28.26,28.26,https://www.drogariaveracruz.com.br/rosucor-10...
396,Benerva 300mg 30 comprimidos revestidos,Valeant Farm Do Brasil,7893454714325,None,None,35.22,30.45,30.45,4.77,4.77,https://www.drogariaveracruz.com.br/benerva-30...
397,Soanza XR 35mg 60 comprimidos,Ache,7896658038457,None,None,98.22,80.25,80.25,17.97,17.97,https://www.drogariaveracruz.com.br/soanza-xr-...
398,Resolor 1mg 14 comprimidos revestidos,Laboratórios Bagó,7896212424269,None,None,165.07,144.27,144.27,20.80,20.80,https://www.drogariaveracruz.com.br/resolor-1m...


💾 Salvando arquivo CSV...
✅ Arquivo salvo como 'veracruz_otimizado.csv'

📈 ESTATÍSTICAS DE QUALIDADE:
Total de produtos: 400
Produtos com preço unitário: 400
Produtos com preço desconto: 400
Produtos com preço PIX: 400
Produtos com marca: 393
Produtos com GTIN: 400

⚡ ESTATÍSTICAS DE PERFORMANCE:
Tempo total: 48.9s
Páginas processadas: 20
Produtos por página (média): 20.0
Tempo por página (média): 2.4s
Requests em cache: 421
Taxa de erro final: 0

📋 Exemplo de produto com dados completos:
  Nome: Blephagel 40g
  Marca: União Química
  GTIN: 3662042000249
  Preco_unitario: 129.9
  Preco_com_desconto: 109.83
  Preco_pix: 109.83
  Desconto: 20.07
  Desconto_com_pix: 20.07
  Link: https://www.drogariaveracruz.com.br/gel-para-higiene-dos-olhos-blephagel-40g/p

🏆 COMPARATIVO DE PERFORMANCE:
Versão original: ~60s estimados (sequencial)
Versão otimizada: 48.9s reais
Melhoria de velocidade: 1.2x mais rápido!
Concurrent operations: 4 pages × 8 products = 32 simultâneos
